In [23]:
#imputs
start_date = "02/01/20 00:00" #start with a Sunday
weeks = 4
station_id = "404888"
Sun = "off"
Mon = "off"
Tues = "on"
Weds = "on"
Thurs = "on"
Fri = "off"
Sat = "off"
Holidays = "off"
frequency = "hour" #enter either "5min", "week", "hour", "day", or "month"

In [24]:
# enter path for where to store output
path = "/Users/clairecasey/Documents/PeMS_data_pull/data/pre_covid/" + station_id +".csv"

In [25]:
import time
import datetime
import requests
from bs4 import BeautifulSoup

In [26]:
from secrets import username, password

In [27]:
#convert dates to UNIX timestamp to be used in URL
start_date_unix = time.mktime(datetime.datetime.strptime(start_date, "%m/%d/%y %H:%M").timetuple())

#convert time to PST time
start_date_pst1 = start_date_unix -25200 
start_date_pst2 = int(start_date_pst1)

weeks_total = list(range(1, weeks+1))
output_body = []

for week_num in weeks_total:
    start_date_pst3 = start_date_pst2 + (3600*24*(week_num-1)*7)
    end_date_pst = start_date_pst3 + (3600*24*((week_num*7)-1)) + (3600*23) #adding 23 hours so end date is 23:59
    start_date_pst4 = str(start_date_pst3)
    end_date_pst1 = str(end_date_pst)

    REQUEST_URL = 'http://pems.dot.ca.gov/?report_form=1&dnode=VDS&content=loops&export=&station_id=' + station_id + '&s_time_id=' + start_date_pst4 + '&s_time_id_f=06%2F04%2F2019+00%3A00&e_time_id=' + end_date_pst1 + '&e_time_id_f=06%2F10%2F2019+23%3A59&tod=all&tod_from=0&tod_to=0&dow_0='+Sun+'&dow_1='+Mon+'&dow_2='+Tues+'&dow_3='+Weds+'&dow_4='+Thurs+'&dow_5='+Fri+'&dow_6='+Sat+'&holidays='+Holidays+'&q=flow&q2=speed&gn=' + frequency + '&agg=on&lane1=on&lane2=on&lane3=on&lane4=on&lane5=on&lane6=on&html.x=52&html.y=14'

    #This URL will be the URL that your login form points to with the "action" tag.
    login_url = 'http://pems.dot.ca.gov/'

    payload = {
        'username': username,
        'password': password
    }
    #get table from url. Can change the class if you wanted to grab other elements from the webpage
    with requests.Session() as session:
        post = session.post(login_url, data=payload)
        s = session.get(REQUEST_URL)

        soup = BeautifulSoup(s.text, 'html.parser')
        table = soup.find('table',class_='inlayTable')
    #create header line for CSV output
    header = table.thead.text
    lines = header.split("\n")
    output_header = [line for line in lines if line.strip() != "" and line.strip() != 'Data Quality']

    #create body array for CSV output
    body = table.tbody.text
    body_short = body[1:(len(body)-2)]
    lines_list = body_short.split("\n\n\n")

    for row in lines_list:
        array = row.split("\n")
        output_body.append(array)



http://pems.dot.ca.gov/?report_form=1&dnode=VDS&content=loops&export=&station_id=404888&s_time_id=1580518800&s_time_id_f=06%2F04%2F2019+00%3A00&e_time_id=1581120000&e_time_id_f=06%2F10%2F2019+23%3A59&tod=all&tod_from=0&tod_to=0&dow_0=off&dow_1=off&dow_2=on&dow_3=on&dow_4=on&dow_5=off&dow_6=off&holidays=off&q=flow&q2=speed&gn=hour&agg=on&lane1=on&lane2=on&lane3=on&lane4=on&lane5=on&lane6=on&html.x=52&html.y=14
72
http://pems.dot.ca.gov/?report_form=1&dnode=VDS&content=loops&export=&station_id=404888&s_time_id=1581123600&s_time_id_f=06%2F04%2F2019+00%3A00&e_time_id=1582329600&e_time_id_f=06%2F10%2F2019+23%3A59&tod=all&tod_from=0&tod_to=0&dow_0=off&dow_1=off&dow_2=on&dow_3=on&dow_4=on&dow_5=off&dow_6=off&holidays=off&q=flow&q2=speed&gn=hour&agg=on&lane1=on&lane2=on&lane3=on&lane4=on&lane5=on&lane6=on&html.x=52&html.y=14
216
http://pems.dot.ca.gov/?report_form=1&dnode=VDS&content=loops&export=&station_id=404888&s_time_id=1581728400&s_time_id_f=06%2F04%2F2019+00%3A00&e_time_id=1583539200&e_

In [20]:
    #write header and body to csv file
    import csv
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(output_header)
        writer.writerows(output_body)